In [5]:
import sys 
sys.path.insert(1,"../src/")
# ! ls "../src/c"
from custom_ort_functions import revert_quantAvgPool
from qonnx.core.modelwrapper import ModelWrapper

model = revert_quantAvgPool(ModelWrapper("../models/mobilenetv1-w4a4_pre_post_tidy.onnx"))
for n in model.graph.node:
    if n.op_type == "AveragePool":
        print(n.attribute)
model.save("../models/mobilenetv1-w4a4_pre_post_tidy_pre_streamline.onnx")

[name: "kernel_shape"
ints: 7
ints: 7
type: INTS
, name: "strides"
ints: 1
ints: 1
type: INTS
]


In [2]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
from custom_steps import (step_mobilenet_streamline)

final_out_dir = "../temp"
model_file = "../models/mobilenetv1-w4a4_pre_post_tidy_pre_streamline.onnx"
folding_config_file = "../folding_config/auto_build_folding.json"

cfg = build.DataflowBuildConfig(
    output_dir          = final_out_dir,
    steps               = [step_mobilenet_streamline],
    mvau_wwidth_max     = 80,
    target_fps          = 50,
    synth_clk_period_ns = 10.0,
    folding_config_file = folding_config_file,
    board               = "KV260_SOM",
    shell_flow_type     = build_cfg.ShellFlowType.VIVADO_ZYNQ,
    generate_outputs=[
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
    ]
)

build.build_dataflow_cfg(model_file, cfg)

Building dataflow accelerator from ../models/mobilenetv1-w4a4_pre_post_tidy_pre_streamline.onnx
Intermediate outputs will be generated in /workspace/results
Final outputs will be generated in ../temp
Build log is at ../temp/build_dataflow.log
Running step: step_mobilenet_streamline [1/1]
Completed successfully


0

In [3]:
! cp "../temp/intermediate_models/step_mobilenet_streamline.onnx" "../models/mobilenet_streamline.onnx"